# Ollama basics

## Models 

- Phi3: context window 8k
- nomic: 8k context 768 dimensions

## Imports

In [ ]:
using System.Net.Http;
using System.Text.Json;


## HTTP Client

In [ ]:
var client = new HttpClient();

## Embeddings

In [ ]:
record EmbeddingResponse(double[] embedding);
async Task<double[]> Embed(string prompt, string model = "nomic-embed-text", string endpoint="http://localhost:11434/api/embeddings")
{
    var jsonStr = JsonSerializer.Serialize(new { prompt, model });
    var response = await client.PostAsync(endpoint, new StringContent(jsonStr, System.Text.Encoding.UTF8, "application/json"));
    if (response.IsSuccessStatusCode)
    {
        var responseStr = await response.Content.ReadAsStringAsync();
        var embedding = JsonSerializer.Deserialize<EmbeddingResponse>(responseStr);
        return embedding.embedding;
    }
    return [];
}

In [ ]:
var vector = await Embed("The quick brown fox jumps over the lazy dog");
vector

## Chat completion

In [ ]:
record Message(string role, string content);
record CompletionResponse(Message message);

async Task<string> Completion(string prompt, string model = "phi3", string endpoint="http://localhost:11434/api/chat")
{   
    var messages = new Message[] { new Message("user", prompt) };    
    var jsonStr = JsonSerializer.Serialize(new { model,messages,stream=false });
    Console.WriteLine(jsonStr);
    var response = await client.PostAsync(endpoint, new StringContent(jsonStr, System.Text.Encoding.UTF8, "application/json"));
    if (response.IsSuccessStatusCode)
    {
        var responseStr = await response.Content.ReadAsStringAsync();
        var completion = JsonSerializer.Deserialize<CompletionResponse>(responseStr);
        return completion.message.content;
    }
    return "Failure";
}

In [ ]:
var completion = await Completion("What is the meaning of life?");
completion